In [1]:
import clingo

In [2]:
def combine(encoding_path, benchmark_path, destination_path):
    
    combined_path = destination_path + "/combined.lp"
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    benchmark_file = open(benchmark_path, "r")
    benchmark = benchmark_file.read()
    benchmark_file.close()
    
    combined_file = open(combined_path, "w")
    combined = combined_file.write(benchmark + encoding)
    combined_file.close()

In [3]:
def solve(robot_id, destination_path, restrictions):
    
    combined_path = destination_path + "/combined.lp"
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    
    for i in range(1,100):
        solution = ""
        horizon = "#const horizon = {}.".format(i)
        asp = horizon + "\n" + rid + "\n" + combined
        if restrictions:
            for i in restrictions:
                if i:
                    asp = asp + "\n" + i
            
        ctl = clingo.Control()
        ctl.add("base", [], asp)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle: solution=("{} ".format(m))
            handle.get()
        solution = solution.replace(' ', '. ')
        
        if solution:
            return solution
            break

In [4]:
def collisions(robot_id, encoding_path, plan):
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    asp = rid + "\n" + plan + "\n" + encoding
    solution = ""
    
    ctl = clingo.Control()
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: solution=("{} ".format(m))
        handle.get()
    if solution== " ":
        return
    else:
        solution = solution.replace(' ', '. ')
        return solution


In [5]:
def plan(robots, destination, collision, restrictions=[]):
    plans=[]
    plan_=""
    actions = []
    
    if not actions:
        for i in robots:
            actions.append([])

    for i in robots:
        actions[i-1].append(solve(i, destination, restrictions))
        if actions[i-1][0]:
            plan_ = plan_ + actions[i-1][0]
            
    cur_rest = []
    for i in robots:
        restrictions = []
        cur_rest.append(collisions(i, collision, plan_))
        if cur_rest[0]:
            restrictions.append(cur_rest[-1])
            plans.append(plan(robots, destination, collision, restrictions))
            
        else:
            return plan_
    return plans

In [10]:
enc = "./asprilo-encodings/shortest_path/pathfinder.lp"
ben = "./instances/abstraction1/instance.lp"
dest = "./python"
col = "./asprilo-encodings/collision/collision.lp"
robots = []
for i in range(1,3):
    robots.append(i)

rest=[]    
plans =[]

In [11]:
combine(enc, ben, dest)

In [12]:
plans=plan(robots, dest, col, rest)

In [13]:
for i in plans:
    print(i)

position(1,(1,3),0). position(1,(1,3),1). position(1,(2,3),2). position(1,(3,3),3). position(1,(4,3),4). position(1,(5,3),5). position(2,(3,1),0). position(2,(3,2),1). position(2,(3,3),2). position(2,(3,4),3). position(2,(3,5),4). 
position(1,(1,3),0). position(1,(2,3),1). position(1,(3,3),2). position(1,(4,3),3). position(1,(5,3),4). position(2,(3,1),0). position(2,(3,1),1). position(2,(3,2),2). position(2,(3,3),3). position(2,(3,4),4). position(2,(3,5),5). 
